In [1]:
# Goal: put data into .png format straight from OG
# kinematic map files

In [41]:
# Okay so now that you have downloaded everything it is time to search through your directory for maps
# and assembly a list of existing data in order to run an analysis
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pyfits
from PIL import Image
from marvin.tools.maps import Maps


# You can also use os to get you the names of all directories in a folder
#print(os.listdir())
# This will yield a tuple for each subdirectory. Ths first entry in the 3-tuple is a directory name, so:

plates = [x[0] for x in os.walk('/Users/beckynevin/sas/mangawork/manga/spectro/analysis/v2_5_3/2.3.0/VOR10-MILESHC-MILESHC/')]
#print(plates)



existing_plateifus=[]

for j in range(len(plates)):
    if re.split('/',plates[j])[-2]=='VOR10-MILESHC-MILESHC':# or re.split('/',plates[j])[-1] != '12701':
        continue
    else:
        existing_plateifus.append(str(re.split('/',plates[j])[-2])+'-'+str(re.split('/',plates[j])[-1]))
print(existing_plateifus)
print('this is how many galaxies you are working with', len(existing_plateifus))
# Now you need to construct plateifu names of what exists

['7990-1902', '9031-6103', '8313-1902', '8314-12703', '8314-1901', '9490-6102', '10497-3702', '9868-6104', '9868-9101', '8724-12704', '8982-9101', '8723-3703', '8723-9101', '8949-12704', '9892-9102', '10499-3701', '10499-12703', '9091-3702', '8947-3704', '8978-3701', '8978-9102', '8315-3704', '9491-12702', '8312-9102', '8713-12704', '9090-1902', '10498-3704', '8983-3704', '8977-3703', '8984-12704', '8341-3702', '8725-1902', '9869-3702', '8657-12704', '10510-9102', '10517-6103', '8454-6101', '10519-9102', '8097-1901', '8256-6104', '8260-1902', '9510-9102', '8455-6103', '8651-9101', '8656-12704', '8259-12705', '8259-3702', '9181-12701', '8257-12704', '8257-3703', '8250-12704', '8616-1901', '9502-3701', '9502-3702', '9505-12704', '8611-12705', '8078-1901', '8244-12705', '8446-9101', '8083-12705', '8077-6104', '8084-12702', '9194-6103', '8274-12703', '9503-3703', '8449-1901', '10503-9101', '8440-6102', '8999-6103', '8999-9101', '8952-6104', '9874-12705', '9041-12701', '8158-12704', '8568-1

In [38]:
# Now grab the stellar velocity maps:

# Here you will assemble a list of the maps from the plateifu list - you could also do this for different
# DAP quantities like other emission lines
# You could also make an array that has multiple maps for each galaxy, but I'll start with one, just
# the stellar velocity maps.
# This section of code stacks them into an array, map_array_ml

# Note that this part takes a little while to run

# First, determine the shape:
shape = 76 # This is I think the maximum size of these cubes

map_array_ml = np.zeros((len(existing_plateifus),shape,shape))#np.zeros((len(plateifu_list),74**2))
map_array_ml_ivar = np.zeros((len(existing_plateifus),shape,shape))#np.zeros((len(plateifu_list),74**2))
map_array_ml_s = np.zeros((len(existing_plateifus),shape,shape))#np.zeros((len(plateifu_list),74**2))
map_array_ml_s_ivar = np.zeros((len(existing_plateifus),shape,shape))#np.zeros((len(plateifu_list),74**2))
#map_array_ml_g = np.zeros((len(existing_plateifus),shape,shape))#np.zeros((len(plateifu_list),74**2))
#map_array_ml_g_ivar = np.zeros((len(existing_plateifus),shape,shape))#np.zeros((len(plateifu_list),74**2))

ifu_name=[]

counter=0
for j in range(len(existing_plateifus)):
    if j%10 == 0: #then j is evenly divisible by 10
        print('j = ',j)
    
    mapit=0
    
    try:
        maps = Maps(filename='/Users/beckynevin/sas/mangawork/manga/spectro/analysis/v2_5_3/2.3.0/VOR10-MILESHC-MILESHC/'+str(re.split('-',existing_plateifus[j])[-2])+'/'+str(re.split('-',existing_plateifus[j])[-1])+'/manga-'+existing_plateifus[j]+'-MAPS-VOR10-MILESHC-MILESHC.fits')
        
        
        
    
        #mapit_g = Maps(filename='/Users/beckynevin/sas/mangawork/manga/spectro/analysis/v2_5_3/2.3.0/VOR10-MILESHC-MILESHC/'+str(re.split('-',existing_plateifus[j])[-2])+'/'+str(re.split('-',existing_plateifus[j])[-1])+'/manga-'+existing_plateifus[j]+'-MAPS-VOR10-MILESHC-MILESHC.fits')['gas_vel'].value
        #mapit_g_ivar = Maps(filename='/Users/beckynevin/sas/mangawork/manga/spectro/analysis/v2_5_3/2.3.0/VOR10-MILESHC-MILESHC/'+str(re.split('-',existing_plateifus[j])[-2])+'/'+str(re.split('-',existing_plateifus[j])[-1])+'/manga-'+existing_plateifus[j]+'-MAPS-VOR10-MILESHC-MILESHC.fits')['gas_vel'].ivar
    
    except:
        maps = Maps(filename='/Users/beckynevin/sas/mangawork/manga/spectro/analysis/v2_5_3/2.3.0/VOR10-MILESHC-MILESHC/'+str(re.split('-',existing_plateifus[j])[-2])+'/'+str(re.split('-',existing_plateifus[j])[-1])+'/manga-'+existing_plateifus[j]+'-MAPS-VOR10-MILESHC-MILESHC.fits.gz')
    
    
    #mask_v = maps['stellar_vel_mask'].value
    mapit = maps['stellar_vel'].value
    mapit_ivar = maps['stellar_vel'].ivar

    #mask_s = maps['stellar_sigma_mask'].value
    mapit_s = np.sqrt(maps['stellar_sigma'].value**2 - maps['stellar_sigmacorr'].value**2)
    mapit_s_ivar = maps['stellar_sigma'].ivar

    '''plt.clf()
    plt.imshow(maps['stellar_sigma'].value, cmap='magma')
    plt.colorbar()
    plt.title('sigma masked uncorrected')
    plt.show()

    plt.clf()
    plt.imshow(maps['stellar_sigmacorr'].value, cmap='magma')
    plt.colorbar()
    plt.title('correction')
    plt.show()

    plt.clf()
    plt.imshow(mapit_s, cmap='magma')
    plt.colorbar()
    plt.title('sigma masked corrected')
    plt.show()
    
    STOP'''
    
    #mapit = Maps(existing_plateifus[j], mode='local', bintype='VOR10', data_origin='db')['stellar_vel'].value
    
    #if plateifu_list[j][-5:-1]=='1270':
        
    try:
        map_array_ml[j,:,:] = mapit
        map_array_ml_ivar[j,:,:] = mapit_ivar
        map_array_ml_s[j,:,:] = mapit_s
        map_array_ml_s_ivar[j,:,:] = mapit_s_ivar
        #map_array_ml_g[j,:,:] = mapit_g
        #map_array_ml_g_ivar[j,:,:] = mapit_g_ivar
        
        
    except: # If this doesn't work you have to pad it first
        # First, determine the difference in shape to make up
        try:
            shape_here = np.shape(mapit)[0]
        
            pad_total = shape - shape_here

            padded_array = np.pad(mapit, (int(pad_total/2),int(pad_total/2)), 'edge')
            padded_array_ivar = np.pad(mapit_ivar, (int(pad_total/2),int(pad_total/2)), 'edge')
            padded_array_s = np.pad(mapit_s, (int(pad_total/2),int(pad_total/2)), 'edge')
            padded_array_s_ivar = np.pad(mapit_s_ivar, (int(pad_total/2),int(pad_total/2)), 'edge')
            #padded_array_g = np.pad(mapit_g, (int(pad_total/2),int(pad_total/2)), 'edge')
            #padded_array_g_ivar = np.pad(mapit_g_ivar, (int(pad_total/2),int(pad_total/2)), 'edge')
            
        except:
            
            STOP
            
        map_array_ml[j,:,:] = padded_array
        map_array_ml_ivar[j,:,:] = padded_array_ivar
        map_array_ml_s[j,:,:] = padded_array_s
        map_array_ml_s_ivar[j,:,:] = padded_array_s_ivar
        #map_array_ml_g[j,:,:] = padded_array_g
        #map_array_ml_g_ivar[j,:,:] = padded_array_g_ivar
        
    ifu_name.append(existing_plateifus[j])
    
    
    # You can set a limit using the below commented out code if you do not need a big training set:
    #if counter > 200:
    #    break
# I have found that it hangs after ~220 galaxies (Brainerror - too many requests for url),
# so you would have to have a dedicated server to store the maps and not do it remotely 
# OR you would have to use a different method to grab the maps.



j =  0


[WARNING]: invalid value encountered in sqrt (RuntimeWarning)


j =  10
j =  20
j =  30
j =  40
j =  50
j =  60
j =  70
j =  80
j =  90
j =  100
j =  110
j =  120
j =  130
j =  140
j =  150
j =  160
j =  170
j =  180
j =  190


In [39]:
print(np.shape(map_array_ml))

(198, 76, 76)


In [40]:
# What about also saving a numpy file so if you want to slightly chage the image you can do so?
# This could also be a better alternative to downloading and saving all of the maps files

for j in range(np.shape(map_array_ml)[0]):

    # also save the array:
    try:
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_vel.npy",
            map_array_ml[j])
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_vel_ivar.npy",
            map_array_ml_ivar[j])
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_disp.npy",
            map_array_ml_s[j])
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_disp_ivar.npy",
            map_array_ml_s_ivar[j])
    except FileNotFoundError: #i.e., if the directory hasn't been created yet
        os.system("mkdir /Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j]))
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_vel.npy",
            map_array_ml[j])
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_vel_ivar.npy",
            map_array_ml_ivar[j])
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_disp.npy",
            map_array_ml_s[j])
        np.save("/Users/beckynevin/CfA_Code/Kinematic_ML/data/numpy_files/"+str(ifu_name[j])+"/stellar_disp_ivar.npy",
            map_array_ml_s_ivar[j])
        
    
    #